# Lab9

In [1]:
### IMPORTS
import numpy as np
import matplotlib
import scipy
from scipy import misc
from sklearn.feature_extraction import image

matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

In [2]:
### parametrii
p = 8 # dimensiunea unui patch (numar de pixeli)
s = 6 # sparsitatea
N = 1000 # numarul total de patch-uri
n = 256 # numarul de atomi din dictionar
K = 50 # numarul de iteratii DL
sigma = 0.075 # deviatia standard a zgomotului


### Exercitiul 1

In [3]:
### a
X = misc.face(gray=True)
m1, m2 = X.shape
# print(X.shape)

# plt.imshow(X, cmap=plt.cm.gray)
# plt.show()
### b

Xnoisy = X + sigma*np.random.randn(m1, m2)
# plt.imshow(Xnoisy, cmap=plt.cm.gray)
# plt.show()


### c
# creez patchuri
Ynoisy = image.extract_patches_2d(Xnoisy, patch_size=(p, p))
# print(Ynoisy[0])
print(Ynoisy.shape)

plt.imshow(Ynoisy[1], cmap=plt.cm.gray)
plt.show()

(773937, 8, 8)


In [4]:
# vectorizam patchurile
Ynoisy = Ynoisy.reshape(Ynoisy.shape[0], -1)
# print(Ynoisy[0])
print(Ynoisy.shape)

(773937, 64)


In [5]:
## matricea transpusa
# Ynoisy = Ynoisy.T
print(Ynoisy.shape)
Ynoisy = np.transpose(Ynoisy)
print(Ynoisy.shape)
# print(Ynoisy[0])

(773937, 64)
(64, 773937)


In [6]:
media_liniilor = []
for i in range(Ynoisy.shape[0]):
    media_liniilor.append(np.mean(Ynoisy[i]))
media_liniilor = np.array(media_liniilor)
# print(media_liniilor.shape)
for i in range(Ynoisy.shape[0]):
    Ynoisy[i] -= media_liniilor[i]

print(Ynoisy[0])
# plt.imshow(Ynoisy, cmap=plt.cm.gray)
# plt.show()
# media_patchurilor = np.mean(media_liniilor)
# print(media_patchurilor)

[ 0.1529551  16.06894111 31.21472039 ... 27.24923309 26.17778161
 32.16067269]


In [7]:
# d
# selectam random N patchuri
Y = []
choices = np.random.choice(Ynoisy.shape[1], N)
for line in Ynoisy:
    Y.append(np.array(line[choices]))
Y = np.array(Y)
print(Y.shape)
# print(Y[0])

(64, 1000)


### Exercitiul 2

In [8]:
from sklearn.preprocessing import normalize
from dictlearn import DictionaryLearning

# Antrenarea dictionarului

D0 = np.random.randn(p * p, n)
D0 = normalize(D0, axis=0, norm='max')

dl = DictionaryLearning(
    n_components=n,
    max_iter=K,
    fit_algorithm='ksvd',
    n_nonzero_coefs=s,
    code_init=None,
    dict_init=D0,
    params=None,
    data_sklearn_compat=False
)
dl.fit(Y)
D= dl.D_

C:\Users\allex\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), OrthogonalMatchingPursuit())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\allex\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler

### Exercitiul 3

In [9]:
from dictlearn import methods

# calcul reprezentare rara a unui semnal
Xc, err = methods.omp(Ynoisy, D, n_nonzero_coefs=s)

C:\Users\allex\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), OrthogonalMatchingPursuit())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(

KeyboardInterrupt



In [ ]:
from sklearn.feature_extraction.image import reconstruct_from_patches_2d

Yc = np.dot(D, Xc)

for i in range(Ynoisy.shape[0]):
    Yc[i] += media_liniilor[i]

Yc = Yc.reshape(8, 8, Yc.shape[1])
# print(Yc.shape)

Yc = np.transpose(Yc, [2, 0, 1])
# print(Yc.shape)

Ic = reconstruct_from_patches_2d(Yc, image_size=(m1, m2))

plt.imshow(Ic, cmap='gray')
plt.show()

In [ ]:
def psnr(original, noisy):
    mse = np.mean((original - noisy) ** 2)
    if mse == 0:
        return np.inf
    max = np.max(original)
    psnr = (max ** 2 ) / mse
    psnr_db = 10 * np.log10(psnr)
    return psnr_db

In [ ]:
print(psnr(X, Ynoisy))
print(psnr(X, Ic))